## Importing used libraries

In [13]:
import numpy as np
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from utils.csvoperations import read_labels

## Defining each path

In [14]:
data_path = 'model/data/landmarks_data.csv'
labels_path = 'model/data/gestures_labels.csv'
model_save_path = 'model/gesture_classifier.hdf5'
tflite_model_save_path = 'model/gesture_classifier.tflite'

## Defining constant values

In [15]:
NUM_CLASSES = 5
N_FEATURES = 21 * 2
RANDOM_STATE = 111

## Loading the data

In [16]:
X_dataset = np.loadtxt(data_path, np.float32, delimiter=',', usecols=list(range(1, N_FEATURES + 1)))

In [17]:
Y_dataset = np.loadtxt(data_path, np.int32, delimiter=',', usecols=[0])

In [18]:
labels = read_labels(labels_path)
print(labels)

{0: 'Open', 1: 'Closed', 2: 'Okay', 3: 'Thumbs up', 4: 'Thumbs down'}


## Data integrity checks

In [19]:
print(f"Number of data samples: {X_dataset.shape[0]}")
print(f"Number of labels: {Y_dataset.shape[0]}")
real_n_classes = len(set(Y_dataset))
print(f"Number of unique classes: {real_n_classes}")
assert real_n_classes == NUM_CLASSES, f"Number of unique classess ({real_n_classes}) differs from NUM_CLASSES ({NUM_CLASSES})"

Number of data samples: 10370
Number of labels: 10370
Number of unique classes: 5


## Creating train and test datasets

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(
                                        X_dataset,
                                        Y_dataset,
                                        test_size=0.2,
                                        random_state=RANDOM_STATE,
                                        stratify=Y_dataset
)

## Building model

In [21]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(N_FEATURES,)),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
Total params: 1125 (4.39 KB)
Trainable params: 1125 (4.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Add model saving callback

In [23]:
checkpoing_callback = keras.callbacks.ModelCheckpoint(
    model_save_path,
    save_weights_only=False,
    verbose=0
)

## Compile and train the model

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test), callbacks=[checkpoing_callback])

Epoch 1/50


2024-05-19 15:41:41.473367: I external/local_xla/xla/service/service.cc:168] XLA service 0x5639590fea70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-19 15:41:41.473422: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2024-05-19 15:41:41.537804: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1716126101.659011   70636 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


260/260 [==============================] - 5s 9ms/step - loss: 1.2858 - accuracy: 0.5259 - val_loss: 1.0249 - val_accuracy: 0.6996
Epoch 2/50
 17/260 [>.............................] - ETA: 1s - loss: 1.0730 - accuracy: 0.6360

/home/acodered/.pyenv/versions/3.10.0/envs/gestures/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


260/260 [==============================] - 2s 9ms/step - loss: 0.9426 - accuracy: 0.6604 - val_loss: 0.7178 - val_accuracy: 0.7917
Epoch 3/50
260/260 [==============================] - 2s 8ms/step - loss: 0.7540 - accuracy: 0.7097 - val_loss: 0.5588 - val_accuracy: 0.8534
Epoch 4/50
260/260 [==============================] - 2s 9ms/step - loss: 0.6581 - accuracy: 0.7583 - val_loss: 0.4500 - val_accuracy: 0.8751
Epoch 5/50
260/260 [==============================] - 2s 8ms/step - loss: 0.5603 - accuracy: 0.7942 - val_loss: 0.3735 - val_accuracy: 0.8954
Epoch 6/50
260/260 [==============================] - 2s 8ms/step - loss: 0.4973 - accuracy: 0.8204 - val_loss: 0.3041 - val_accuracy: 0.9209
Epoch 7/50
260/260 [==============================] - 2s 8ms/step - loss: 0.4329 - accuracy: 0.8479 - val_loss: 0.2405 - val_accuracy: 0.9349
Epoch 8/50
260/260 [==============================] - 2s 8ms/step - loss: 0.3921 - accuracy: 0.8567 - val_loss: 0.2157 - val_accuracy: 0.9368
Epoch 9/50
260/26

In [26]:
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

65/65 [==============================] - 0s 6ms/step - loss: 0.0887 - accuracy: 0.9778
Test accuracy: 0.98


In [27]:
model = keras.models.load_model(model_save_path)

In [28]:
_, _ = model.evaluate(X_train, Y_train)

260/260 [==============================] - 1s 5ms/step - loss: 0.0718 - accuracy: 0.9832


In [29]:
result = model.predict(np.array([X_test[0]]))
print(result)
print(f"{np.argmax(result)} - {labels[np.argmax(result)]}")

1/1 [==============================] - 0s 47ms/step
[[9.9983525e-01 6.1941535e-11 6.6216926e-05 9.0094409e-13 9.8505356e-05]]
0 - Open


## Convert model to Tensorflow Lite

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open(tflite_model_save_path, 'wb') as file:
    file.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_yu6ewkd/assets


INFO:tensorflow:Assets written to: /tmp/tmp_yu6ewkd/assets
2024-05-19 15:43:39.098999: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-19 15:43:39.099057: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-19 15:43:39.099320: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_yu6ewkd
2024-05-19 15:43:39.100061: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-19 15:43:39.100073: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_yu6ewkd
2024-05-19 15:43:39.102774: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-05-19 15:43:39.135274: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp_yu6ewkd
2024-05-19 15:43:39.144473: I tensorflow/cc/saved_model/loader.cc:316] SavedModel